# Tasks

A good first reference for most things in Julia is by referencing the documentation. You can do this by typing `?` followed by the keyword you'd like to refer. 

Since our first discussion is on Tasks, let us load the documentation for a `Task`:

In [1]:
?Task

search: Task task_local_storage @task istaskdone istaskstarted current_task



```
Task(func)
```

Create a `Task` (i.e. coroutine) to execute the given function (which must be callable with no arguments). The task exits when this function returns.

```jldoctest
julia> a() = det(rand(1000, 1000));

julia> b = Task(a);
```

In this example, `b` is a runnable `Task` that hasn't started yet.


A Julia Task is: 

- a very lightweight coroutine
- Not a thread!
- Internal to and scheduled by a Julia Process

In [1]:
function mytask()
    println("Going to take a nap.")
    sleep(10)
    println("Woke up.")
    rand()
end

t=Task(mytask)

Task (runnable) @0x0000000120b5ad10

What happened here? We've created a task just like how the documentation told us. But is it running? 

The task is currently a `runnable`, which means that it is _created_ but not _scheduled_ yet. 

## Scheduling and waiting on a task

`schedule` starts the task, but will *return immediately*. This means that it does **not** block the master process.

(**NOTE**: Run the next two cells immediately one after the other before looking at the accompanying text)

In [2]:
schedule(t)

Going to take a nap.


Task (runnable) @0x0000000120b5ad10

## Waiting on a task

The task has now been scheduled and is actively running in the background. Since it hasn't blocked the master process, we can perform some computation in the meantime. 

In [3]:
println("Doing something else while t is taking a nap...")
inv(rand(100, 100))
@time @show wait(t)
@show t.state
println("task finished")

Doing something else while t is taking a nap...
Woke up.
wait(t) = 0.3386435289376857
  4.109741 seconds (15.08 k allocations: 857.336 KiB)
t.state = :done
task finished


## `@async` - syntax sugar for creating and scheduling tasks

Of course, we can **create and schedule tasks in one go** by putting code in an `@async` block

In [6]:
t=@async begin
    println("Going to take a nap.")
    sleep(5)
    println("Woke up.")
end

Going to take a nap.


Task (runnable) @0x00000001214cf610

Sure enough, before the 5 seconds of sleep time are up, we can schedule computation. 

In [7]:
21+21

42

Woke up.


## Channels

Channels are used for communication between Tasks. To demonstrate, consider the following simple producer-consumer model, like so:

In [8]:
input = Channel{Int}(1)
result = Channel{Int}(1)
doubler = @async while true
    x = take!(input)
    println("Got message $x")
    put!(result, 2x)
end

printer = @async while true
    res = take!(result)
    @show res
end

Task (runnable) @0x000000011efec490

Now let's add some input to the `Channel` via the `put!` command.

In [10]:
using Interact
@manipulate for i=1:100
    put!(input, i)
end

Interact.Options{:SelectionSlider,Any}(1: "input" = 50 Any , "i", 50, "50", 50, Interact.OptionDict(DataStructures.OrderedDict{Any,Any}("1"=>1,"2"=>2,"3"=>3,"4"=>4,"5"=>5,"6"=>6,"7"=>7,"8"=>8,"9"=>9,"10"=>10…), Dict{Any,Any}(Pair{Any,Any}(68, "68"),Pair{Any,Any}(2, "2"),Pair{Any,Any}(89, "89"),Pair{Any,Any}(11, "11"),Pair{Any,Any}(39, "39"),Pair{Any,Any}(46, "46"),Pair{Any,Any}(85, "85"),Pair{Any,Any}(25, "25"),Pair{Any,Any}(55, "55"),Pair{Any,Any}(42, "42")…)), Any[], Any[], true, "horizontal")

Got message 50
res = 100


50

Got message 46
res = 92
Got message 65
res = 130
Got message 64
res = 128
Got message 62
res = 124
Got message 57
res = 114
Got message 47
res = 94
Got message 40
res = 80
Got message 34
res = 68
Got message 30
res = 60
Got message 31
res = 62
Got message 35
res = 70
Got message 40
res = 80


## Adding Julia Processes, running "Remote Tasks"

Now let's start running tasks remotely, on other Julia processes. First, we need to request our cluster manager (`JuliaRun`) for 4 worker Julia processes. Note that this means that the **master** process can now **shedule work** on the 4 worker processes. 

In [ ]:
using JuliaRunClient
ctx = Context()
nb = self()

In [ ]:
initParallel()
@result setJobScale(ctx, nb, 4)
waitForWorkers(4)

If you were on your own laptop or on a cluster that isn't set up with `JuliaRun`, you should use the `addprocs` command to initialize Julia worker processes.

In [29]:
# Run if using the notebook on your own computer
# addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

Since we have a master process and 4 worker processes, the total number of processes we have initialized is 5. 

In [30]:
procs()

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

Now let's consider a simple example to demonstrate the use of these new tools. 



## Estimate pi in parallel

There's a simple monte carlo method one can use to calculate $\pi$: 
1. Remember that the ratio of the area of a unit circle and a unit square is: 
$$ 4r^2 / \pi r^2 = \pi / 4$$ where $r$ is the radius of the circle.
2. Next, remember that the square of the coordinates of a point gives you the distance from the origin. 
3. We can now randomly simulate `N` points, and calculate the fraction of points that fall within the unit circle. 
4. This is the ratio of the area of a unit circle and unit square. 4 times this ratio gives you the value of $\pi$.

In [31]:
@everywhere function trials(numsteps=1000)  # default value of the parameter
    pos = 0 
    for j in 1:numsteps
        pos += Int(rand()^2 + rand()^2 < 1)
    end
    return pos
end

function estimate_pi(in_circle, N)
    4in_circle / N
end

estimate_pi (generic function with 1 method)

Let's see if it works with 10^8 trials.

In [32]:
estimate_pi(trials(10^8), 10^8)

3.14129036

## `@spawnat` - schedule tasks on different procceses 

`@spawnat` is like @async but runs on a different process

In [33]:

f=@spawnat 3 begin
    println("Process ", myid(), " starting random trials")
    res = trials(10^8)
    println("Process ", myid(), " done")
    res
end

Future(3, 1, 10, Nullable{Any}())

	From worker 3:	Process 3 starting random trials
	From worker 3:	Process 3 done


In [34]:
typeof(f)

Future

What's the curious `Future(3,1,12,Nullable{Any}())` thing?

In [35]:
f[]

78532694

A `Future` is a reference to the computation on a Julia worker (aka remote) process. Doing `f[]` returns its values

Now our monte carlo simulation to estimate $\pi$ is embarrassingly parallel, so we can offload some of the computation to another Julia process. Just like we created a task using `@async`, but this time it's running a task on a remote process.

In [36]:
function remote_trials(pid,n)
    @spawnat pid begin
        println("Process ", myid(), " starting trials")
        trials(n)
    end
end

remote_trials (generic function with 1 method)

For example, let us schedule 1000 trials on process 2. Since this task is scheduled on another process, it returns a `Future`. 

In [37]:
remote_trials(2, 1000)

Future(2, 1, 12, Nullable{Any}())

	From worker 2:	Process 2 starting trials


Therefore, to estimate $\pi$ parallel, we need to spawn trials on all our worker processes.  

In [38]:
function parallel_trials(n, pids=workers())
    @time futures = [remote_trials(p,n) for p in pids]
    sum([f[] for f in futures])
end

parallel_trials (generic function with 2 methods)

Each of them would start a number of trials and return the number of trials that fell within the unit circle. Eventually, we divide by the total number of trials, and estimate the value of $\pi$. Let us see if our simulation works. 

In [39]:
@time estimate_pi(parallel_trials(10^8), 10^8*nworkers())

	From worker 2:	Process 2 starting trials
  0.025225 seconds (1.32 k allocations: 73.993 KiB)
	From worker 3:	Process 3 starting trials
	From worker 5:	Process 5 starting trials
	From worker 4:	Process 4 starting trials
  1.573160 seconds (50.47 k allocations: 2.902 MiB, 0.70% gc time)


3.14163855

(Remember to run it twice to get the true time!)

Let us compare with time in serial:

In [40]:
@time estimate_pi(trials(10^8*nworkers()), 10^8*nworkers())

  1.884120 seconds (18 allocations: 384 bytes)


3.14155212